In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [11]:
import requests
import numpy as np
import cv2

def get_ipcam_frame(url):
    try:
        response = requests.get(url, timeout=2)
        if response.status_code != 200:
            return None

        img_array = np.frombuffer(response.content, np.uint8)
        if img_array.size == 0:
            return None

        frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        return frame

    except Exception:
        return None


In [7]:
from collections import deque

PRED_HISTORY = 7
conf_buffer = deque(maxlen=PRED_HISTORY)
class_buffer = deque(maxlen=PRED_HISTORY)


In [8]:
import cv2
import json
import numpy as np
import tensorflow as tf
from collections import deque

# ===============================
# CONFIG
# ===============================
MODEL_PATH = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/best_face_model.keras"
LABEL_PATH = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/class_labels.json"

IP_CAM_STREAM = "http://192.168.1.30:4747/video"

IMG_SIZE = 112
CONF_THRESHOLD = 0.70

FRAME_WIDTH = 640
FRAME_HEIGHT = 480
TARGET_FPS = 10
FRAME_SKIP = 2

ROTATE_FRAME = True
ROTATE_MODE = cv2.ROTATE_90_COUNTERCLOCKWISE  # Adjust if needed

# Temporal smoothing config
PRED_HISTORY = 7
MIN_STABLE_COUNT = 4

# ===============================
# LOAD CLASS LABELS
# ===============================
with open(LABEL_PATH, "r") as f:
    labels = json.load(f)

if all(k.isdigit() for k in labels.keys()):
    class_names = {int(k): v for k, v in labels.items()}
else:
    class_names = {v: k for k, v in labels.items()}

# ===============================
# LOAD MODEL
# ===============================
model = tf.keras.models.load_model(MODEL_PATH)

# ===============================
# FACE DETECTOR
# ===============================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ===============================
# VIDEO CAPTURE
# ===============================
cap = cv2.VideoCapture(IP_CAM_STREAM)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
cap.set(cv2.CAP_PROP_FPS, TARGET_FPS)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

if not cap.isOpened():
    print("Camera open failed")
    exit(1)

print("Press 'q' to quit")

# ===============================
# TEMPORAL BUFFERS
# ===============================
conf_buffer = deque(maxlen=PRED_HISTORY)
class_buffer = deque(maxlen=PRED_HISTORY)

frame_count = 0

# ===============================
# MAIN LOOP
# ===============================
while True:
    ret, frame = cap.read()
    if not ret:
        print("Frame read failed")
        break

    # Rotate frame if needed
    if ROTATE_FRAME:
        frame = cv2.rotate(frame, ROTATE_MODE)

    frame_count += 1

    # Skip frames for performance
    if frame_count % FRAME_SKIP != 0:
        cv2.imshow("Face Recognition Test", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(80, 80)
    )

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        face = face.astype("float32") / 255.0
        face = np.expand_dims(face, axis=0)

        preds = model.predict(face, verbose=0)[0]
        class_id = int(np.argmax(preds))
        confidence = float(preds[class_id])

        # Append to temporal buffers
        conf_buffer.append(confidence)
        class_buffer.append(class_id)

        avg_conf = np.mean(conf_buffer)
        stable_class = max(set(class_buffer), key=class_buffer.count)

        # Stable prediction logic
        if avg_conf >= CONF_THRESHOLD and class_buffer.count(stable_class) >= MIN_STABLE_COUNT:
            name = class_names.get(stable_class, "Unknown")
            label = f"{name} ({avg_conf*100:.1f}%)"
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(
            frame,
            label,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.75,
            color,
            2
        )

    cv2.imshow("Face Recognition Test", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# ===============================
# CLEANUP
# ===============================
cap.release()
cv2.destroyAllWindows()


Camera open failed
Press 'q' to quit
Frame read failed


[tcp @ 0x24e87d00] Connection to tcp://192.168.1.30:4747 failed: Connection timed out


In [10]:
import os
import json
import numpy as np
import tensorflow as tf
import cv2

# ================== CONFIG ==================
MODEL_PATH = r"/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/best_face_model.keras"  # Change path
LABEL_PATH = r"/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/class_labels.json"  # Change path
IMG_SIZE = 112  # Must match your training
CONF_THRESHOLD = 0.5
DOWNLOAD_FOLDER = r"/home/sandeshprasai/Downloads"  # Change path if needed

# ================== LOAD MODEL ==================
# Uncomment the below line if you want to force CPU (in case GPU causes errors)
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded successfully")

# ================== LOAD LABELS ==================
with open(LABEL_PATH, "r") as f:
    labels = json.load(f)

# Correct mapping: index -> class_name
class_names = {int(k): v for k, v in labels.items()}
print(f"Loaded {len(class_names)} class labels")

# ================== HELPER FUNCTION ==================
def predict_image(image_path):
    """Load image, preprocess, predict, return class + confidence"""
    if not os.path.exists(image_path):
        print(f"❌ File does not exist: {image_path}")
        return None

    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Failed to read image: {image_path}")
        return None

    # Resize & normalize
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img, verbose=0)[0]
    class_id = int(np.argmax(preds))
    confidence = float(preds[class_id])

    if confidence >= CONF_THRESHOLD:
        name = class_names.get(class_id, "Unknown")
    else:
        name = "Unknown"

    return name, confidence

# ================== TEST IMAGES ==================
image_files = [f for f in os.listdir(DOWNLOAD_FOLDER)
               if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp"))]

print(f"Found {len(image_files)} images in {DOWNLOAD_FOLDER}")

for img_file in image_files:
    path = os.path.join(DOWNLOAD_FOLDER, img_file)
    result = predict_image(path)
    if result:
        name, conf = result
        print(f"{img_file} -> {name} ({conf*100:.2f}%)")


Model loaded successfully
Loaded 540 class labels
Found 7 images in /home/sandeshprasai/Downloads
0007_01.jpg -> n000148 (99.94%)
download.png -> n000273 (53.26%)
0008_01.jpg -> n000010 (100.00%)
download1.png -> n000008 (99.77%)
download.jpeg -> n000008 (99.95%)
0005_01.jpg -> n000273 (100.00%)
0013_01.jpg -> n000008 (99.96%)


In [14]:
model = tf.keras.models.load_model(MODEL_PATH)

# Create embedding model (output = dense_1)
embedding_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer("dense_1").output
)

# ================== IMAGE PREPROCESS ==================
def preprocess_image(image_path, img_size=112):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# ================== GENERATE EMBEDDINGS ==================
embeddings_dict = {}

for filename in os.listdir(DOWNLOAD_FOLDER):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(DOWNLOAD_FOLDER, filename)
        img = preprocess_image(img_path)
        embedding_vector = embedding_model.predict(img, verbose=0)
        # Normalize embedding for similarity tasks
        embedding_vector = embedding_vector / np.linalg.norm(embedding_vector)
        embeddings_dict[filename] = embedding_vector.flatten()

print(f"Generated embeddings for {len(embeddings_dict)} images.")

# ================== SAVE EMBEDDINGS ==================
import numpy as np
np.save(os.path.join(DOWNLOAD_FOLDER, "face_embeddings.npy"), embeddings_dict)
print("✅ Embeddings saved to face_embeddings.npy")

Generated embeddings for 7 images.
✅ Embeddings saved to face_embeddings.npy


In [ ]:
import numpy as np

data = np.load("/home/sandeshprasai/Downloads/face_embeddings.npy", allow_pickle=True).item()
print(data.keys())  # list all images loaded
print(data["download1.png"])  # example embedding


In [23]:
import numpy as np

# embeddings for two images
emb1 = data["0007_01.jpg"]
emb2 = data["0013_01.jpg"]

# normalize vectors
emb1 = emb1 / np.linalg.norm(emb1)
emb2 = emb2 / np.linalg.norm(emb2)

# cosine similarity
similarity = np.dot(emb1, emb2)
print("Cosine similarity:", similarity)


Cosine similarity: 0.24210279
